In [158]:
import sys
import os

# Get the path of the current notebook's directory and append the parent directory to the path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

import importlib

import pdc_functions.assumption_utils as assumptions
import pdc_functions.data_utils as data
import pdc_functions.explore_utils as explore
import pdc_functions.pdc_1and2_utils as pdc
import pdc_functions.stats_utils as stats
from dotenv import load_dotenv
import pdc_functions.class_utils as class_utils
import pandas as pd


from phmlondon.snow_utils import SnowflakeConnection


FileNotFoundError: [Errno 2] No such file or directory

In [197]:
importlib.reload(stats) # Use when updating the functions

<module 'pdc_functions.stats_utils' from '/Users/jsaund/Documents/repos/onelondon_snowflake_datascience/projects/meds_compliance/pdc_functions/stats_utils.py'>

In [ ]:
# Snowflake env needed
load_dotenv()

snowsesh = SnowflakeConnection()
snowsesh.use_database("INTELLIGENCE_DEV")
snowsesh.use_schema("AI_CENTRE_FEATURE_STORE")

In [ ]:
# Uses function to select all from pre made snowflake table. See sql folder for the sql used to generate the cohort tables
# ldl_all table is a table of all the ldl measurements (using the open codelists ldl measurements definition ID
ldl = data.get_data_by_cohort(snowsesh, "intelligence_dev.ai_centre_dev.ldl_all")


Clean the ldl column first - discussed with Dan - awaiting pipeline updates

In [179]:
print(ldl)

         person_id age_at_event result_date  \
0          1627994        35.17  2016-06-23   
1          2310420        61.50  2013-04-29   
2           701442        85.42  2020-08-12   
3          3496476        65.67  2015-08-22   
4          1760514        29.50  2006-03-30   
...            ...          ...         ...   
8775923    1461878        58.17  2022-06-09   
8775924    2398877        55.92  2016-05-04   
8775925     602269        77.50  2025-02-17   
8775926     775129        71.17  2017-03-06   
8775927     387765        63.08  2024-05-20   

                                        code_description  result_value  \
0        Serum low density lipoprotein cholesterol level          3.59   
1        Serum low density lipoprotein cholesterol level          4.42   
2        Serum low density lipoprotein cholesterol level          1.10   
3        Serum low density lipoprotein cholesterol level          3.30   
4        Serum low density lipoprotein cholesterol level         

In [ ]:
# Uses function to select all from pre made snowflake table. See sql folder for the sql used to generate the cohort tables
# Statin_all table is a table of all orders of a Lipid-regulating drug as per the BNF class definition 
statin = data.get_data_by_cohort(snowsesh, "intelligence_dev.ai_centre_dev.statin_all")

In [182]:
print(statin['person_id'].nunique())
print(ldl['person_id'].nunique())

273736
1501296


In [ ]:
# This uses a medicationtable class to ensure all the orders information is in the table and in the correct format
drug_table = class_utils.MedicationTable(statin)


In [ ]:
# This uses the functions in the medicationstable class.
drug_table.validate_columns()  # Validate columns
drug_table.normalise_missing_values()  # Standardise representation of missing data
drug_table.convert_dates()  # Convert date columns
drug_table.validate_data_types()  # Validate data types
drug_table.clean_dose()  # Clean and interpret the 'dose' column
drug_table.calculate_covered_days()

print("All validations passed successfully!")

Invalid rows (dose or tablets_per_day issue):
Empty DataFrame
Columns: [dose, tablets_per_day]
Index: []
All validations passed successfully!


In [ ]:
# Compute overal pdc calsulate the inclusive and exclusive pdcs per person per drug in the time frame available
# this add the one row per order table so the pdc's will be the sae for the same person/drug combo
person_drug_pdc = pdc.compute_pdc_overall(drug_table.df)

/Users/jsaund/Documents/repos/onelondon_snowflake_datascience/meds_env/lib/python3.11/site-packages/numpy/_core/fromnumeric.py:57: RuntimeWarning: overflow encountered in multiply
  return bound(*args, **kwds)
/Users/jsaund/Documents/repos/onelondon_snowflake_datascience/meds_env/lib/python3.11/site-packages/numpy/_core/fromnumeric.py:57: RuntimeWarning: overflow encountered in multiply
  return bound(*args, **kwds)
/Users/jsaund/Documents/repos/onelondon_snowflake_datascience/meds_env/lib/python3.11/site-packages/numpy/_core/fromnumeric.py:57: RuntimeWarning: overflow encountered in multiply
  return bound(*args, **kwds)
/Users/jsaund/Documents/repos/onelondon_snowflake_datascience/meds_env/lib/python3.11/site-packages/numpy/_core/fromnumeric.py:57: RuntimeWarning: overflow encountered in multiply
  return bound(*args, **kwds)
/Users/jsaund/Documents/repos/onelondon_snowflake_datascience/meds_env/lib/python3.11/site-packages/numpy/_core/fromnumeric.py:57: RuntimeWarning: overflow enco

Running checks - yay works.

Now, if there are any duration days that are negative - these are ignored when calculating covered_days

In [173]:
print(person_drug_pdc[person_drug_pdc['total_covered_days'] < 0])


Empty DataFrame
Columns: [person_id, drug_name, min_start_date, max_start_date, gender, ethnicity, imd, age_at_start, overall_inclusive_pdc, overall_exclusive_pdc, total_covered_days, total_exposed_days]
Index: []


In [167]:
# adds demographic data - updated with the new IMD source.

person_drug_pdc = data.add_demographic_data(snowsesh, person_drug_pdc)

Merged data: 24136543 rows with columns: ['person_id', 'age_at_event', 'order_date', 'drug_name', 'dose', 'duration_days', 'quantity_unit', 'quantity_value', 'tablets_per_day', 'calculated_duration', 'order_enddate', 'covered_days', 'overall_inclusive_pdc', 'overall_exclusive_pdc', 'total_covered_days', 'total_exposure_days', 'date_of_birth', 'gender', 'ethnicity', 'imd']


In [168]:
# this then aggregates the table into one row per person per drug including columns like 
person_drug_pdc = data.general_agg(person_drug_pdc)

In [169]:
print(person_drug_pdc)

        person_id                  drug_name min_start_date max_start_date  \
0               3                Simvastatin     2002-10-17     2014-08-01   
1               5                Simvastatin     2013-05-01     2013-05-01   
2               6                  Ezetimibe     2004-07-20     2013-04-02   
3               6                Simvastatin     2004-05-25     2004-06-23   
4               9                Simvastatin     2006-12-01     2015-11-25   
...           ...                        ...            ...            ...   
415817    6482933              Colestyramine     1994-09-29     1994-09-29   
415818    6483863               Atorvastatin     2015-03-24     2024-10-21   
415819    6485822               Atorvastatin     2020-01-17     2025-04-01   
415820    6486132               Ciprofibrate     2007-07-06     2023-03-29   
415821    6489132  Omega-3-acid ethyl esters     2009-02-13     2009-07-16   

        gender    ethnicity  imd  age_at_start  overall_inclusi

Join to ldl results table

In [174]:
# this results attachment add the 3 closest results within a +/- a year of the drug start and drug end dates for each person/drug combo
data_joined = data.attach_closest_results(person_drug_pdc,ldl)

In [175]:
print(data_joined)


        person_id                  drug_name min_start_date max_start_date  \
0               3                Simvastatin     2002-10-17     2014-08-01   
1               5                Simvastatin     2013-05-01     2013-05-01   
2               6                  Ezetimibe     2004-07-20     2013-04-02   
3               6                Simvastatin     2004-05-25     2004-06-23   
4               9                Simvastatin     2006-12-01     2015-11-25   
...           ...                        ...            ...            ...   
415817    6482933              Colestyramine     1994-09-29     1994-09-29   
415818    6483863               Atorvastatin     2015-03-24     2024-10-21   
415819    6485822               Atorvastatin     2020-01-17     2025-04-01   
415820    6486132               Ciprofibrate     2007-07-06     2023-03-29   
415821    6489132  Omega-3-acid ethyl esters     2009-02-13     2009-07-16   

        gender    ethnicity  imd  age_at_start  overall_inclusi

In [176]:
# Whereas this attchement selects the 3 results ONLY before the start date and ONLY after the end date
data_joined_2 =  data.attach_closest_results_2(person_drug_pdc,ldl)

Clean 
- remove people without at least a year of medication
- remove people with missing bloods - need at least 2 before and 2 after for average


Need to check the results with data_joined_2

In [177]:
clean_data = data.cohort_exclusions(data_joined)

In [178]:
print(clean_data)

        person_id                  drug_name min_start_date max_start_date  \
0               3                Simvastatin     2002-10-17     2014-08-01   
4               9                Simvastatin     2006-12-01     2015-11-25   
5              10                Simvastatin     2008-06-18     2015-02-26   
7              14                Simvastatin     2009-10-28     2013-07-01   
9              15                Simvastatin     2007-12-17     2019-09-27   
...           ...                        ...            ...            ...   
415805    6412440                Simvastatin     2011-02-15     2012-12-11   
415812    6433901  Omega-3-acid ethyl esters     2014-09-24     2020-04-22   
415818    6483863               Atorvastatin     2015-03-24     2024-10-21   
415819    6485822               Atorvastatin     2020-01-17     2025-04-01   
415820    6486132               Ciprofibrate     2007-07-06     2023-03-29   

        gender    ethnicity  imd  age_at_start  overall_inclusi

Average the results before/after

In [183]:
# Creates a avg_before and avg_after col with a result_diff col per person per drug
analysis_data = data.avg_results(clean_data)
print(analysis_data)

        person_id                  drug_name min_start_date max_start_date  \
0               3                Simvastatin     2002-10-17     2014-08-01   
4               9                Simvastatin     2006-12-01     2015-11-25   
5              10                Simvastatin     2008-06-18     2015-02-26   
7              14                Simvastatin     2009-10-28     2013-07-01   
9              15                Simvastatin     2007-12-17     2019-09-27   
...           ...                        ...            ...            ...   
415805    6412440                Simvastatin     2011-02-15     2012-12-11   
415812    6433901  Omega-3-acid ethyl esters     2014-09-24     2020-04-22   
415818    6483863               Atorvastatin     2015-03-24     2024-10-21   
415819    6485822               Atorvastatin     2020-01-17     2025-04-01   
415820    6486132               Ciprofibrate     2007-07-06     2023-03-29   

        gender    ethnicity  imd  age_at_start  overall_inclusi

EXPLORE THE ANALYSIS TABLE

In [184]:
print(analysis_data.describe())


          person_id                 min_start_date  \
count  1.421980e+05                         142198   
mean   1.552283e+06  2009-08-12 12:14:49.531498496   
min    3.000000e+00            1990-09-17 00:00:00   
25%    6.184780e+05            2006-07-19 00:00:00   
50%    1.319821e+06            2008-10-21 00:00:00   
75%    2.394865e+06            2012-08-23 18:00:00   
max    6.486132e+06            2024-04-29 00:00:00   
std    1.117514e+06                            NaN   

                      max_start_date   age_at_start  overall_inclusive_pdc  \
count                         142198  142198.000000          140775.000000   
mean   2015-04-18 19:58:36.555788544      61.626000               0.728885   
min              1991-10-28 00:00:00       8.000000               0.000000   
25%              2012-03-30 00:00:00      52.000000               0.447689   
50%              2014-12-15 00:00:00      62.000000               0.792969   
75%              2018-11-07 00:00:00      71.

In [187]:
nan_counts = analysis_data.isna().sum()
print(nan_counts)

person_id                    0
drug_name                    0
min_start_date               0
max_start_date               0
gender                       0
ethnicity                 2549
imd                          0
age_at_start                 0
overall_inclusive_pdc     1423
overall_exclusive_pdc     1423
total_covered_days           0
total_exposed_days        1423
before_result_1           3196
after_result_1            1587
before_result_2          39851
after_result_2           50622
before_result_3          85422
after_result_3           95618
before_avg                   0
after_avg                    0
result_diff                  0
dtype: int64


Errors:
- negative and massive total covered days?
- leading to negative and massive overall_inclusive_pd
- results ldl max can be >100?!
- some poeple on medication for 79 years....?
  
Plan
- look at and ?remove rows where total covered days are negative - DONE
- ldl needs cleaning - await dan
- 

In [ ]:
# Unadjusted Linear regression



need to remoe the rows with no exlusive pdc

In [203]:
df_clean_1 = analysis_data.dropna(subset=["overall_exclusive_pdc", "result_diff"])

linr_unadj = stats.linear_reg(df_clean_1, "overall_exclusive_pdc", "result_diff")

In [190]:
print(linr_unadj.summary())

                            OLS Regression Results                            
Dep. Variable:            result_diff   R-squared:                       0.001
Model:                            OLS   Adj. R-squared:                  0.001
Method:                 Least Squares   F-statistic:                     84.45
Date:                Thu, 22 May 2025   Prob (F-statistic):           3.99e-20
Time:                        16:53:39   Log-Likelihood:            -3.6722e+05
No. Observations:              140775   AIC:                         7.344e+05
Df Residuals:                  140773   BIC:                         7.345e+05
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                    -0.34

In [191]:
categorical_cols = ['gender', 'ethnicity', 'drug_name', 'imd']
outcome_col = 'result_diff'

# Prepare the data
df_clean = stats.prepare_data_for_regression(analysis_data, categorical_cols, outcome_col)


In [192]:
print(df_clean.columns)

Index(['person_id', 'min_start_date', 'max_start_date', 'age_at_start',
       'overall_inclusive_pdc', 'overall_exclusive_pdc', 'total_covered_days',
       'total_exposed_days', 'before_result_1', 'after_result_1',
       'before_result_2', 'after_result_2', 'before_result_3',
       'after_result_3', 'before_avg', 'after_avg', 'result_diff',
       'gender_Male', 'gender_Other', 'gender_Unknown',
       'ethnicity_Any other Asian background',
       'ethnicity_Any other Black background',
       'ethnicity_Any other White background',
       'ethnicity_Any other ethnic group',
       'ethnicity_Any other mixed background', 'ethnicity_Bangladeshi',
       'ethnicity_British', 'ethnicity_Caribbean', 'ethnicity_Chinese',
       'ethnicity_Indian', 'ethnicity_Irish', 'ethnicity_Not stated',
       'ethnicity_Pakistani', 'ethnicity_White and Asian',
       'ethnicity_White and Black African',
       'ethnicity_White and Black Caribbean', 'drug_name_Atorvastatin',
       'drug_name_Bezafi

In [198]:
exclude_cols = [outcome_col, 'person_id', 'min_start_date', 'max_start_date', 'overall_inclusive_pdc', 'total_covered_days', 'total_exposed_days', 'before_result_1', 'after_result_1',
       'before_result_2', 'after_result_2', 'before_result_3',
       'after_result_3', 'before_avg', 'after_avg', 'result_diff']
covariates = [col for col in df_clean.columns if col not in exclude_cols]


linr_adj = stats.linear_reg(df_clean, covariates, outcome_col)

print(linr_adj.summary())



                            OLS Regression Results                            
Dep. Variable:            result_diff   R-squared:                       0.013
Model:                            OLS   Adj. R-squared:                  0.011
Method:                 Least Squares   F-statistic:                     6.738
Date:                Thu, 22 May 2025   Prob (F-statistic):           3.65e-42
Time:                        16:55:38   Log-Likelihood:                -40624.
No. Observations:               24221   AIC:                         8.135e+04
Df Residuals:                   24172   BIC:                         8.174e+04
Df Model:                          48                                         
Covariance Type:            nonrobust                                         
                                             coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------

In [200]:
print(df_clean.dtypes)

person_id                         int64
min_start_date           datetime64[ns]
max_start_date           datetime64[ns]
age_at_start                      int64
overall_inclusive_pdc           float64
                              ...      
imd_6.0                           int64
imd_7.0                           int64
imd_8.0                           int64
imd_9.0                           int64
imd_nan                           int64
Length: 65, dtype: object


In [201]:
print(linr_adj.summary())

                            OLS Regression Results                            
Dep. Variable:            result_diff   R-squared:                       0.013
Model:                            OLS   Adj. R-squared:                  0.011
Method:                 Least Squares   F-statistic:                     6.738
Date:                Thu, 22 May 2025   Prob (F-statistic):           3.65e-42
Time:                        16:57:22   Log-Likelihood:                -40624.
No. Observations:               24221   AIC:                         8.135e+04
Df Residuals:                   24172   BIC:                         8.174e+04
Df Model:                          48                                         
Covariance Type:            nonrobust                                         
                                             coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------